In [ ]:
# Setup chunk to install and load required packages
knitr::opts_chunk$set(warning = FALSE, message = FALSE)
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'glmnet',
               'randomForest', 'xgboost','patchwork',
               'paletteer', 'here', 'doParallel', 'summarytools')


## Regression - Optimize and save models

In the previous notebook, you used complex regression models to look at the relationship between features of a bike rentals dataset. In this notebook, you'll see if you can improve the performance of these models even further.

Let's load the bicycle-sharing data as a tibble and view the first few rows. You'll also split the data into training and test datasets.


In [ ]:
# Load the required packages and make them available in your current R session
suppressPackageStartupMessages({
  library(tidyverse)
  library(tidymodels)
  library(lubridate)
  library(paletteer)
})

# Import the data into the R session
bike_data <- read_csv(file = "https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/data/daily-bike-share.csv", show_col_types = FALSE)

# Parse dates then extract days
bike_data <- bike_data %>%
  mutate(dteday = mdy(dteday)) %>% 
  mutate(day = day(dteday))

# Select desired features and labels
bike_select <- bike_data %>% 
  select(c(season, mnth, holiday, weekday, workingday, weathersit,
           temp, atemp, hum, windspeed, rentals)) %>% 
  mutate(across(1:6, factor))

# Split 70% of the data for training and the rest for testing
set.seed(2056)
bike_split <- bike_select %>% 
  initial_split(prop = 0.7,
  # splitting data evenly on the holiday variable
                strata = holiday)

# Extract the data in each split
bike_train <- training(bike_split)
bike_test <- testing(bike_split)

# Specify multiple regression metrics
eval_metrics <- metric_set(rmse, rsq)


cat("Training Set", nrow(bike_train), "rows",
    "\nTest Set", nrow(bike_test), "rows")


The result is two datasets:

-   **bike_train**: A subset of the dataset used to train the model.
-   **bike_test**: A subset of the dataset used to validate the model.

Now you're ready to train a model by fitting a boosting ensemble algorithm, as in the last notebook. Recall that a gradient boosting estimator is like a random forest algorithm. Instead of building all the trees independently and taking the average result, each tree is built on the outputs of the previous one. This technique is an attempt to incrementally reduce the loss (error) in the model.


In [ ]:
# For reproducibility
set.seed(2056)

# Build an xgboost model specification
boost_spec <- boost_tree() %>% 
  set_engine('xgboost') %>% 
  set_mode('regression')

# Train an xgboost model 
boost_mod <- boost_spec %>% 
  fit(rentals ~ ., data = bike_train)

# Make and bind predictions to test data a
results <- boost_mod %>% 
  augment(new_data = bike_test) %>% 
  rename(predictions = .pred)

# Evaluate the model
boost_metrics <- eval_metrics(data = results,
                                  truth = rentals,
                                  estimate = predictions) 

# Plot predicted vs actual
boost_plt <- results %>% 
  ggplot(mapping = aes(x = rentals, y = predictions)) +
  geom_point(color = '#4D3161FF') +
  # overlay regression line
  geom_smooth(method = 'lm', color = 'black', se = F) +
  ggtitle("Daily Bike Share Predictions") +
  xlab("Actual Labels") +
  ylab("Predicted Labels") +
  theme(plot.title = element_text(hjust = 0.5))

# Return evaluations
list(boost_metrics, boost_plt)


### Preprocess the data by using recipes

You trained a model with data that was loaded straight from a source file, with only moderately successful results. In practice, it's common to perform some preprocessing of the data to make it easier for the algorithm to fit a model to it.

In this section, you'll explore the Tidymodels package [recipes](https://tidymodels.github.io/recipes/). This package helps you preprocess your data before you train your model. A recipe is an object that defines a series of steps for data processing.

There are many preprocessing transformations you can perform to get your data ready for modeling. Let's limit the options to a few common techniques.

#### Scale numeric features

Numeric features need to be normalized so that they're on the same scale. Normalizing prevents features with large values from producing coefficients that disproportionately affect the predictions. For example, suppose your data includes the following numeric features:

|  A  |  B  |  C  |
|:---:|:---:|:---:|
|  3  | 480 | 65  |

Normalizing these features to the same scale might result in the following values. You're assuming that A contains values from 0 to 10, B contains values from 0 to 1,000, and C contains values from 0 to 100:

|  A  |  B   |  C   |
|:---:|:----:|:----:|
| 0.3 | 0.48 | 0.65 |

There are multiple ways you can scale numeric data. You can calculate the minimum and maximum values for each column and assign a proportional value between 0 and 1. You can also use the mean and standard deviation of a normally distributed variable to maintain the same spread of values on a different scale.

#### Encode categorical variables

Encoding involves translating a column with categorical values into one or more numeric columns that take the place of the original.

Machine learning models work best with numeric features rather than text values. Generally, you need to convert categorical features into numeric representations. For example, suppose your data includes the following categorical feature:

| Size |
|:----:|
|  S   |
|  M   |
|  L   |

You can apply ordinal encoding to substitute a unique integer value for each category, as shown:

| Size |
|:----:|
|  0   |
|  1   |
|  2   |

Another common technique is to create *dummy* or *indicator variables*. They replace the original categorical feature with numeric columns whose values are either 1 or 0, as shown:

| Raw data |  M  |  L  |
|:--------:|:---:|:---:|
|    S     |  0  |  0  |
|    M     |  1  |  0  |
|    L     |  0  |  1  |

In R, the convention is to exclude a column for the first factor level. In this case, the factor is *S*. The reasons for this include *simplicity* and reducing *linear dependencies*. The full set of encodings can also be used for some models. This technique is traditionally called *one-hot* encoding and can be achieved by using the `one_hot` argument of `step_dummy()`.

> Tree-based models created by using the XGBoost engine typically require one to create dummy variables.

Now, let's create some recipes.


In [ ]:
# Specify a recipe
bike_recipe <- recipe(rentals ~ ., data = bike_train) %>% 
  step_normalize(all_numeric_predictors()) %>% 
  step_dummy(all_nominal_predictors()) 

# Print out recipe
bike_recipe


# Summary of the recipe
summary(bike_recipe)


You just created your first recipe that contains an outcome and its corresponding predictors. The numeric predictors are normalized. The nominal predictors are converted to a quantitative format. Let's quickly explain:

-   The call to `recipe()` with a formula tells the recipe the roles of the variables. Examples are predictor and outcome. The `bike_train` data is used as the reference. See the results of `summary(bike_recipe)`.
-   The `step_normalize(all_numeric_predictors())` function specifies that all numeric predictors should be normalized.
-   The `step_dummy(all_nominal_predictors())` function specifies that all predictors that are currently factor or character should be converted to a quantitative format (1s/0s).

Now that you have a recipe, the next step is to create a model specification. In this case, let's re-create an `xgboost` model specification.


In [ ]:
# xgboost specification
boost_spec <- boost_tree() %>% 
  set_engine('xgboost') %>% 
  set_mode('regression')


But how do you combine this model specification with the data preprocessing steps from your recipe?

You'll use modeling workflows, which are known as pipelines in Python.

### Bundle it all together by using a workflow

The [workflows](https://workflows.tidymodels.org/) package allows you to bind modeling and preprocessing objects together. You can then fit the entire workflow to the data so that the model encapsulates all the preprocessing steps and the algorithm.


In [ ]:
# Create the workflow
boost_workflow <- workflow() %>% 
  add_recipe(bike_recipe) %>% 
  add_model(boost_spec)

# Print out workflow
boost_workflow


The workflow object provides an informative summary of the preprocessing steps that will be done by the recipe and the model specification. Also, a `workflow()` can be fit in much the same way a model can.



In [ ]:
# Train the model
boost_workflow <- boost_workflow %>% 
  fit(data = bike_train)

boost_workflow


Now that you have your fitted workflow, how do you make some predictions? You can use `predict()` on a workflow in the same way as you used it on a model.

Let's make some predictions on the first six observations of the test set.


In [ ]:
boost_workflow %>% 
  predict(new_data = bike_test %>% dplyr::slice(1:6))


Workflows are convenient!

Why haven't we made predictions on the whole test set, evaluated performance, and created some graphs? We'll get to that. First, let's address a boosted tree's hyperparameters:


In [ ]:
args(boost_tree)



Those are a lot of model arguments: `mtry`, `trees`, `min_n`, `tree_depth`, `learn_rate`, `loss_reduction`, `sample_size`, and `stop_iter`.

How do you know what values you should use?

That's where model tuning comes in.

### Tune model hyperparameters

Models have parameters with unknown values that must be estimated to use the model for predicting. These model parameters, known as hyperparameters or tuning parameters, can't be learned directly from a dataset during model training. You can estimate the best values by training many models on a simulated dataset. Then you measure how well all these models perform. This process is called tuning.

We won't go into the details of each hyperparameter. But they work together to affect the way the algorithm trains a model. For instance, in boosted trees:

-   The `min_n` argument forces the tree to discard any node that has a number of observations below your specified minimum.
-   Tuning the value of `mtry` controls the number of variables that will be used at each split of a decision tree.
-   Tuning `tree_depth` helps by restricting your tree from growing after it reaches a certain level.

In many cases, the default values provided by Tidymodels will work well. See the defaults by running `help("boost_tree")`. There might be some advantages in modifying hyperparameters to get better predictive performance or reduce training time.

How do you know what hyperparameter values you should use? Without a deep understanding of how the underlying algorithm works, you'll need to experiment. Fortunately, Tidymodels provides a way to tune hyperparameters by trying multiple combinations and finding the best result for a given performance metric.

#### Identify tuning parameters

How can you signal to Tidymodels functions which arguments, such as `cost_complexity`, `tree_depth`, and `min_n`, should be optimized? Parameters are marked for tuning by assigning them a value of `tune()`.

Let's build the model specification with some tuning. Then you'll put your recipe and model specification together in a workflow for ease of use.


In [ ]:
# Specify a recipe
bike_recipe <- recipe(rentals ~ ., data = bike_train) %>% 
  step_normalize(all_numeric_predictors()) %>% 
  step_dummy(all_nominal_predictors()) 


# Make a tunable model specification
boost_spec <- boost_tree(trees = 50,
                         tree_depth = tune(),
                         learn_rate = tune()) %>% 
  set_engine('xgboost') %>% 
  set_mode('regression')


# Bundle a recipe and model spec using a workflow
boost_workflow <- workflow() %>% 
  add_recipe(bike_recipe) %>% 
  add_model(boost_spec)

# Print workflow
boost_workflow


#### Create a tuning grid

Now that you've specified what parameter to tune, you need to figure out a set of possible values to try out. Then you'll choose the best.


In [ ]:
# Create a grid of tuning parameters
tree_grid <- grid_regular(tree_depth(),
                  # You can specify hyperparameter ranges too
learn_rate(range = c(0.01, 0.3), trans = NULL), levels = 5)

# Display hyperparameter combinations that will be used for tuning
tree_grid


The function [`grid_regular()`](https://tidymodels.github.io/dials/reference/grid_regular.html) chooses sensible values to try for each hyperparameter. Here, you asked for five of each. Because you have two hyperparameters to tune, `grid_regular()` returns 5×5 = 25 different possible tuning combinations to try in a tidy tibble format.

#### Sample the data

As we pointed out earlier, hyperparameters can't be learned directly from the training set. Instead, they're estimated by using simulated datasets created from a process called resampling. One resampling approach is cross-validation.

Cross-validation involves taking your training set and randomly dividing it up evenly into *V* subsets or folds. You then use one of the folds for validation and the rest for training. You repeat these steps with all the subsets and combine the results, usually by taking the mean. This process is just one round of cross-validation. To obtain better results, sometimes data scientists cross-validate more than once, perhaps five times.


In [ ]:
set.seed(2056)
# 5 fold CV repeated once
bike_folds <- vfold_cv(data = bike_train, v = 5, repeats = 1)


#### Time to tune

Now it's time to tune the grid to find out which hyperparameter combination results in the best performance.


In [ ]:
# Allow parallel processing
doParallel::registerDoParallel()

# Model tuning via grid search
set.seed(2020)
tree_grid <- tune_grid(
  object = boost_workflow,
  resamples = bike_folds,
  grid = tree_grid
)


#### Visualize tuning results

You have trained models for many possible hyperparameter values, so let's explore the results.

First, you'll use the function `collect_metrics()` to extract the performance metrics from the tuning results.


In [ ]:
# Visualize the results
tree_grid %>% 
  collect_metrics() %>% 
  mutate(tree_depth = factor(tree_depth)) %>% 
  ggplot(mapping = aes(x = learn_rate, y = mean,
                       color = tree_depth)) +
  geom_line(size = 0.6) +
  geom_point(size = 2) +
  facet_wrap(~ .metric, scales = 'free', nrow = 2)+
  scale_color_viridis_d(option = "plasma", begin = .9, end = 0)


Our stubbiest tree, with a depth of 1, is the worst model according to both metrics, rmse and rsq, and across all candidate values of `learn_rate`. A tree depth of 4 and a learn_rate of around 0.1 seem to be best. Let's investigate these tuning parameters further. You can use `show_best()` to display the top sub-models and their performance estimates.



In [ ]:
tree_grid %>% 
  show_best('rmse')


You can then use `select_best()` to find the tuning parameter combination with the best performance values.

In [ ]:
# Select the tree with the best RMSE
best_tree <- tree_grid %>% 
  select_best('rmse')

# Display best tree
best_tree


### Finalize your model

Now that you have the best performance values, you can use `tune::finalize_workflow()` to update, or finalize, your workflow object with the best estimate values for tree_depth and learn_rate.


In [ ]:
# Update workflow
final_wf <- boost_workflow %>% 
  finalize_workflow(best_tree)

# Print final workflow
final_wf


Our tuning is done. You've updated your workflow with the best estimated hyperparameter values.

#### The last fit: Back to the test set

Let's return to the test data and estimate the model performance you expect to see with new data. You can use the function [`last_fit()`](https://tidymodels.github.io/tune/reference/last_fit.html) with your finalized model. This function fits the finalized model on the full training dataset and evaluates the finalized model on the testing data.


In [ ]:
# Make a last fit
final_fit <- final_wf %>% 
  last_fit(bike_split)


# Collect metrics
final_fit %>% 
  collect_metrics()


How's that for a tune? There seems to be some improvement in the evaluation metrics compared to using the default values for learn_rate and tree_depth hyperparameters. Now we leave it to you to explore how tuning the other hyperparameters affects the model performance.

You've seen a number of common techniques used to train predictive models for regression. In a real project, you would likely try a few more algorithms, hyperparameters, and preprocessing transformations. By now you should have  the general idea of the procedure to follow. You can always explore the [reference docs](https://www.tidymodels.org/find/parsnip/#models) or use the `args()` function to see which parsnip object arguments are available.

Let's now explore how you can use the trained model with new data.

### Use the trained model

Before you save your model, let's extract the trained workflow object from the `final_fit` object.


In [ ]:
# Extract trained workflow
bike_boost_model <- final_fit$.workflow[[1]]


You can save this model to be used later.



In [ ]:
# Save trained workflow
saveRDS(bike_boost_model, 'bike_boost_model.rds')


You can load it whenever you need it and use it to predict labels for new data. This technique is often called *scoring* or *inferencing*.

For example, let's try and predict some values from the test set by using the saved model.


In [ ]:
# Load the model
loaded_model <- readRDS("bike_boost_model.rds")

# Extract predictors
bike_new <- bike_test %>% 
  dplyr::slice(5:9)

# Use the model to predict rentals
results <- loaded_model %>% 
  augment(bike_test)

# See model predictions 
results


All is well that ends with a working model.

### Summary

We've concluded the notebooks for this module on regression. In this notebook, you ran a complex regression, tuned it, saved the model, and used it to predict outcomes for the future.

### Further reading

To learn more about Tidymodels, see the [Tidymodels documentation](https://www.tidymodels.org/).
